In [ ]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap, ScalarMappable
from quetzal.model import stepmodel
from quetzal.io import excel

# Validate different scenarios against each other

In [ ]:
# Resolution of output plots
dpi = 600

In [ ]:
scenario_ref = 'reference'
scenarios = [scenario_ref] + ['Avoid', 'Shift', 'Avoid+Shift']

In [ ]:
input_path = '../input/'
network_path = '../input_static/'
output_path = '../output/'
model_path = '../model/'
params = excel.read_var(file='../input/parameters.xls', scenario=scenario_ref)
segments = [s.strip() for s in params['general']['demand_segments'].split(';')]
purposes = [s.split('_')[0] for s in segments][::2]

In [ ]:
ref = stepmodel.read_json(model_path + scenario_ref + '/de_zones')
ref.zones = gpd.GeoDataFrame(ref.zones, crs=ref.epsg)

In [ ]:
# Load distances
distances = pd.read_csv(output_path + '/distances_centroids.csv')
distances = distances.set_index(['origin', 'destination'])

In [ ]:
# Load the calibration dataset for validation
if False:
    mid2017 = pd.read_csv(input_path + 'transport_demand/calibration_all_trips_MiD2017.csv')
    # Replace LAU codes with NUTS IDs
    assert str(mid2017.loc[0, 'origin']).startswith('DE')
    # Rename modes and purposes
    mode_dict_mid = {1: 'rail_short', 2: 'rail_long',
                     3: 'coach', 4: 'bus', 5: 'air', 6: 'car', 7: 'walk'}
    mid2017['mode_model'] = mid2017['mode_model'].map(mode_dict_mid)
    mid2017['purpose_model'] = mid2017['purpose_model'].apply(lambda s: s.split('_')[0])
    mid2017['segment'] = mid2017['purpose_model'] + mid2017['car_avail'].map(
        {1: '_car', 0: '_no_car', 9: '_no_car'})
    mid2017 = mid2017[['mode_model', 'purpose_model', 'segment', 'origin', 'destination']]

# Composite cost

Percieved cost averaged over all modes by origin-destination pair

In [ ]:
# Load CC
cc = {}
for scenario in scenarios:
    cc[scenario] = pd.read_csv(output_path + scenario + '/mode_choice_od_composite_cost.csv')

In [ ]:
# Differences to reference scenario
fig, ax = plt.subplots(nrows=len(scenarios)-1, ncols=3, figsize=(15,(len(scenarios)-1)*5),
                       sharex='all', sharey='all')
label_step = 90
ref_matrix = cc[scenario_ref].set_index(['origin', 'destination']).min(axis=1).unstack('destination')
for i in range(1, len(scenarios)):
    i -= 1
    ax[i,0].set_ylabel(scenario)
    matrix = cc[scenario].set_index(['origin', 'destination']).min(axis=1).unstack('destination')
    ax[i,0].imshow((matrix - ref_matrix).fillna(0).values)
    matrix = cc[scenario].set_index(['origin', 'destination']).mean(axis=1).unstack('destination')
    ax[i,1].imshow((matrix - ref_matrix).fillna(0).values)
    matrix = cc[scenario].set_index(['origin', 'destination']).max(axis=1).unstack('destination')
    ax[i,2].imshow((matrix - ref_matrix).fillna(0).values)
    ylabels = list(matrix.columns)
    ax[i,0].set_yticks(range(0, len(ylabels), label_step), labels=ylabels[::label_step], fontsize=8)
    if i == len(scenarios)-2:
        xlabels = list(matrix.index)
        ax[i,0].set_xticks(range(0, len(xlabels), label_step), labels=xlabels[::label_step], fontsize=8, rotation=90)
        ax[i,1].set_xticks(range(0, len(xlabels), label_step), labels=xlabels[::label_step], fontsize=8, rotation=90)
        ax[i,2].set_xticks(range(0, len(xlabels), label_step), labels=xlabels[::label_step], fontsize=8, rotation=90)
        ax[i,0].set_xlabel('Min')
        ax[i,1].set_xlabel('Mean')
        ax[i,2].set_xlabel('Max')

In [ ]:
# Define distance classes
bins = [0, 20, 40, 60, 80, 100, 150, 200, 300, 500, 700, 1000]
labels = ['{}-{}km'.format(bins[i], bins[i+1]) for i in range(len(bins)-1)]
for scenario in scenarios:
    cc[scenario]['dist'] = cc[scenario].set_index(['origin', 'destination']).index.map(distances.to_dict()['length'])
    cc[scenario]['bins'] = pd.cut(cc[scenario]['dist'], bins=bins, labels=labels)

In [ ]:
fig, ax = plt.subplots(nrows=3, ncols=1, figsize=(15,10),
                       sharex='all', sharey='all')
for scenario in scenarios:
    cc[scenario].loc[cc[scenario].notna().all(axis=1)].groupby('bins')[segments].mean().min(axis=1).plot.line(
        legend=True, label=scenario, ax=ax[0])
    cc[scenario].loc[cc[scenario].notna().all(axis=1)].groupby('bins')[segments].mean().mean(axis=1).plot.line(
        legend=True, label=scenario, ax=ax[1])
    cc[scenario].loc[cc[scenario].notna().all(axis=1)].groupby('bins')[segments].mean().max(axis=1).plot.line(
        legend=True, label=scenario, ax=ax[2])
ax[0].set_ylabel('Min')
ax[1].set_ylabel('Mean')
ax[2].set_ylabel('Max')

In [ ]:
cc = None

# Volumes

Number of trips per year

In [ ]:
# Load volumes
vols = {}
for scenario in scenarios:
    vols[scenario] = stepmodel.read_zippedpickles(model_path + scenario + '/de_volumes')

In [ ]:
# Inner- and inter-zonal volumes by segment
seg_df = pd.DataFrame()
for scenario in scenarios:
    vol = vols[scenario].volumes
    seg_df[scenario+'_inner'] = vol.loc[vol['origin']==vol['destination'], segments].sum()
    seg_df[scenario+'_inter'] = vol.loc[vol['origin']!=vol['destination'], segments].sum()

In [ ]:
# Compare inner- and inter-zonal volumes by segment
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12,4))
seg_df.loc[[s for s in segments if '_no_car' in s]].plot.bar(legend=True, ax=ax[0])
seg_df.loc[[s for s in segments if not '_no_car' in s]].plot.bar(legend=False, ax=ax[1])

In [ ]:
seg_df['reference_inter'].sum() / (seg_df['reference_inner'].sum() + seg_df['reference_inter'].sum())

In [ ]:
# Compare inner- and inter-zonal volumes by segment
# Stacked bar plot: inner+inter by scenario by segment
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6,3.5))
segs = [s for s in segments if '_no_car' in s]+[s for s in segments if not '_no_car' in s]
cmap = get_cmap('magma') # or cividis
colors_inner = {'reference': cmap(.3), 'Avoid': cmap(.5), 'Shift': cmap(.7), 'Avoid+Shift': cmap(.95)}
colors_inter = {'reference': cmap(.25), 'Avoid': cmap(.45), 'Shift': cmap(.65), 'Avoid+Shift': cmap(.9)}
ind = np.arange(len(segments))
width = 1/len(scenarios) - 0.1
width_filled = 0
for scenario in scenarios:
    ax.bar(ind+width_filled-width*1.5, seg_df.loc[segs, scenario+'_inner'].values,
           bottom=0, width=width,
           label=scenario+' inner',
           color=[colors_inner[scenario] for _ in segments])
    ax.bar(ind+width_filled-width*1.5, seg_df.loc[segs, scenario+'_inter'].values,
           bottom=seg_df.loc[segs, scenario+'_inner'].values,
           width=width,
           label=scenario+' inter',
           hatch='////',
           color=[colors_inter[scenario] for _ in segments])
    width_filled += width
ax.set_ylabel('trip frequency')
ax.set_xticks(ind, segs, rotation=90)
h, l = ax.get_legend_handles_labels()
ax.legend(h[::-1], l[::-1])
plt.savefig(output_path + '_scenario_validation/inner-inter_shares.png', dpi=dpi, bbox_inches = "tight")

### Distance distribution

inter-zonal trips

In [ ]:
# Define distance classes
bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 120, 150, 200, 300, 500, 1000]
labels = ['{}-{}km'.format(bins[i], bins[i+1]) for i in range(len(bins)-1)]
for scenario in scenarios:
    vol = vols[scenario].volumes
    vol['dist'] = vol.set_index(['origin', 'destination']).index.map(distances.to_dict()['length'])
    vol['bins'] = pd.cut(vol['dist'], bins=bins, labels=labels)

In [ ]:
# Sum up volumes
vol_df = pd.DataFrame()
for scenario in scenarios:
    vol_df[[scenario+' '+seg for seg in segments]] = vols[scenario].volumes.groupby('bins')[segments].sum()
    vol_df[scenario+' sum'] = vol_df[[scenario+' '+seg for seg in segments]].sum(axis=1)
    vol_df[scenario+' sum with car'] = vol_df[[scenario+' '+seg for seg in segments if not '_no_car' in seg]].sum(axis=1)
    vol_df[scenario+' sum without car'] = vol_df[[scenario+' '+seg for seg in segments if '_no_car' in seg]].sum(axis=1)

In [ ]:
# plot total
ax = vol_df[[s+' sum' for s in scenarios]].plot(kind='area', stacked=False, alpha=.4, colormap=cmap, figsize=(12,4))
ax.set_xlabel(None)
ax.set_ylabel('trip frequency')
plt.savefig(output_path + '_scenario_validation/distance_distribution_total.png', dpi=dpi)

In [ ]:
# plot by car availability
fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(12,4))
vol_df[[s+' sum without car' for s in scenarios]].plot(kind='area', stacked=False, colormap=cmap, alpha=.4, ax=ax[0])
vol_df[[s+' sum with car' for s in scenarios]].plot(kind='area', stacked=False, colormap=cmap, alpha=.4, ax=ax[1])
ax[0].set_xlabel(None)
ax[1].set_xlabel(None)
ax[0].set_ylabel('trip frequency')
ax[0].tick_params(axis="x", rotation=50)
ax[1].tick_params(axis="x", rotation=50)
plt.savefig(output_path + '_scenario_validation/distance_distribution_car_ownership.png', dpi=dpi)

In [ ]:
# plot by segment
fig, ax = plt.subplots(ncols=1, nrows=len(segments), figsize=(12,len(segments)*3), sharex='all')
i = 0
for seg in segments:
    vol_df[[scenario+' '+seg for scenario in scenarios]].plot(kind='area', stacked=False, colormap=cmap, alpha=.4, ax=ax[i])
    i += 1

# Modal split

by main mode. Inter-zonal (i.e. no local transport)

In [ ]:
# Define the colors
colormap = {'car':'#690709', 'private_car':'#690709', 'car_sharing':'#A23133',
            'rail':'#151770', 'rail_long':'#1C1F9D', 'rail_short':'#151770',
            'coach':'#B7A111', 'bus':'#887706',
            'air': '#bdbdbd', 'air_domestic': '#bdbdbd',
            'non-motorised':'#494949'} # also defines mode order

In [ ]:
# Load probabilities and merge with volumes to create an OD matrix
ods = {}
for scenario in scenarios:
    sm = stepmodel.read_zippedpickles(model_path + scenario + '/de_logit')
    od = sm.probabilities.set_index(['origin', 'destination', 'segment'])\
        .drop(columns=['root'], errors='ignore').unstack('segment')
    modes = set(od.columns.get_level_values(0))
    if len(['rail' for mode in modes if 'rail' in mode]) > 1:
        od.drop('rail', axis=1, inplace=True)
    if len(['car' for mode in modes if 'car' in mode]) > 1:
        od.drop('car', axis=1, inplace=True)
    vol = vols[scenario].volumes.loc[
        vols[scenario].volumes['origin']!=vols[scenario].volumes['destination']
    ].set_index(['origin', 'destination']).drop(columns=['bins', 'dist', 'index'], errors='ignore')
    for mode in set(od.columns.get_level_values(0)):
        od[mode] = od[mode] * vol
    ods[scenario] = od
    assert np.round(od.sum().sum(), 0) == np.round(vol.sum().sum(), 0), \
        'Mode choice probabilities do not sum up to one per OD pair and segment'
    del sm

In [ ]:
# Modal split total between zones
# Layer thickness represents total volumes between zones
# Multiple ring plot
fig, ax = plt.subplots(figsize=(6,6), layout='tight')
width_sum = sum([od.sum().sum() for od in ods.values()]) * 1.3 # for empty space in the middle
width_filled = 0
for scenario in scenarios[::-1]:
    width = ods[scenario].sum().sum() / width_sum
    values = ods[scenario].stack('segment').sum().T.rename(index={'walk': 'non-motorised'})
    values = values.loc[[mode for mode in colormap.keys() if mode in values.index]]
    pie, _ = ax.pie(values,
                    labels=values.index if scenario==scenarios[-1] else None,
                    radius=1-width_filled,
                    colors=[colormap[mode] for mode in values.index]
                   )
    plt.setp(pie, width=width, edgecolor='white')
    width_filled += width
    text = scenario+' ({}%)'.format(
        round(ods[scenario].sum().sum() / ods[scenario_ref].sum().sum() * 100))
    ax.annotate(text,
                xy=(-len(text)*0.0185, (1-width_filled)+0.06),
                bbox=dict(boxstyle="round", fc="white", lw=None, alpha=.8)
    )
plt.savefig(output_path + '_scenario_validation/modal_split_trips.png', dpi=dpi)

In [ ]:
# Define distance classes
bins = [0, 20, 30, 40, 50, 60, 70, 80, 90, 100, 120, 150, 200, 300, 500, 1000]
labels = ['{}-{}km'.format(bins[i], bins[i+1]) for i in range(len(bins)-1)]
for s in scenarios:
    ods[s]['dist'] = ods[s].merge(distances, how='left', left_index=True, right_index=True)['length']
    ods[s]['bins'] = pd.cut(ods[s]['dist'], bins=bins, labels=labels)
    ods[s].drop('dist', axis=1, inplace=True)

In [ ]:
# Bar plot with distance distribution and mode shares
fig, ax = plt.subplots(nrows=len(segments), ncols=1, figsize=(14,len(segments)*3.5),
                       sharex='all', sharey='row')
width_filled = 0
width = 1/len(scenarios) - 0.1
ind = np.arange(len(labels))
for scenario in scenarios:
    for i in range(len(segments)):
        od = ods[scenario].groupby('bins').sum().swaplevel(0,1, axis=1)[segments[i]].rename(
            columns={'walk': 'non-motorised'})
        modes = [m for m in colormap.keys() if m in od.columns]
        od = od[modes]
        for m in range(len(modes)):
            ax[i].bar(ind+width_filled, od[modes[m]].values,
                      bottom=od.iloc[:, :m].sum(axis=1).values if m>0 else 0,
                      width=width, tick_label=labels,
                      color=colormap[modes[m]],
                      label=modes[m])#+' '+scenario)
        ax[i].set_ylabel(segments[i])
    ax[0].annotate(scenario, xy=(width_filled-width/4, 1e8),
                   rotation=90, fontsize=7.5)
    width_filled += width
h, l = ax[0].get_legend_handles_labels()
ax[0].legend([h[l.index(m)] for m in colormap.keys() if m in l][::-1],
             [m for m in colormap.keys() if m in l][::-1])

In [ ]:
# Bar plot with distance distribution - total
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(14,4))
width_filled = 0
width = 1/len(scenarios) - 0.1
ind = np.arange(len(labels))
for scenario in scenarios:
    od = ods[scenario].stack('segment').fillna(method='bfill').\
        groupby('bins').sum().reindex(labels).rename(
            columns={'walk': 'non-motorised'})
    modes = [m for m in colormap.keys() if m in od.columns]
    od = od[modes]
    for m in range(len(modes)):
        ax.bar(ind+width_filled, od[modes[m]].values,
               bottom=od.iloc[:, :m].sum(axis=1).values if m>0 else 0,
               width=width, tick_label=labels,
               color=colormap[modes[m]],
               label=modes[m])#+' '+scenario)
    ax.annotate(scenario, xy=(width_filled+ind[-1]-width/4, 5e8),
                rotation=90, fontsize=7.5)
    width_filled += width
ax.set_ylabel('Number of trips per year')
h, l = ax.get_legend_handles_labels()
ax.legend([h[l.index(m)] for m in colormap.keys() if m in l][::-1],
          [m for m in colormap.keys() if m in l][::-1])

# Passenger kilometre

pkm = #trips * distance

Take network loads (not main mode)

In [ ]:
# Load pkm
pkm = {}
for s in scenarios:
    df = pd.read_csv(output_path + s + '/pkm.csv')
    df = df.set_index(['origin', 'mode']).unstack('mode')['pkm'].dropna(how='all').fillna(0)
    pkm[s] = df[[mode for mode in df.columns if df[mode].sum()>df.sum().sum()*0.001]]

In [ ]:
# Pkm mode shares
# Pie size represents total pkm
all_sum = sum([pkm[s].sum().sum() for s in scenarios])
fig, ax = plt.subplots(nrows=1, ncols=len(scenarios), figsize=(len(scenarios)*4,4),
                       gridspec_kw={'width_ratios':
                                    [pkm[s].sum().sum() / all_sum for s in scenarios]
                                   })
for i in range(len(scenarios)):
    pkm[scenarios[i]].sum().plot.pie(ax=ax[i], ylabel=scenarios[i])

In [ ]:
# Pkm as bar plot
fig, ax = plt.subplots(figsize=(6,4))
pkm_df = pd.DataFrame()
for s in scenarios:
    pkm_df[s] = pkm[s].sum()
pkm_df.loc['air_domestic', scenario_ref] = 10.4e9
pkm_df = pkm_df.rename(index={'walk': 'non-motorised'}).fillna(0)
modes = [m for m in colormap.keys() if m in pkm_df.index]
pkm_df.T[modes].plot.bar(stacked=True, ax=ax, legend=None,
                         color=[colormap[m] for m in modes])
ax.set_ylabel('pkm/a')
h,l = ax.get_legend_handles_labels()
plt.legend(h[::-1], l[::-1], loc='center left')
plt.savefig(output_path + '_scenario_validation/modal_split_pkm.png', dpi=dpi)

### Energy demand

Energy intensities depend on the transport technology and its propulsion system. There  are the following data sets available:
* TREMOD (Transport Emission Model; underlying values come from HBEFA (Handbuch Emissionsfaktoren)): "Aktualisierung der Modelle TREMOD/TREMOD-MM für die Emissionsberichterstattung 2020 (Berichtsperiode 1990-2018)", 2020
* better overview of TREMOD data for today here: https://www.umweltbundesamt.de/daten/verkehr/endenergieverbrauch-energieeffizienz-des-verkehrs#spezifischer-energieverbrauch-sinkt
* Easy diagrams for today here: https://www.allianz-pro-schiene.de/presse/pressemitteilungen/ferienstart-im-umweltvergleich-liegt-die-bahn-vorn/
* Projections for 2050 (choosing figures for fully electrified drivetrains): Robinius, M. et al. (2020). Kosteneffiziente und Klimagerechte Transformationsstrategien für das deutsche Energiesystem bis zum Jahr 2050. Jülich.
* Maximum efficiency assumptions for 2050: Millward-Hopkins, J., Steinberger, J. K., Rao, N. D., & Oswald, Y. (2020). Providing decent living with minimum energy: A global scenario. Global Environmental Change, 65, 102168. https://doi.org/10.1016/j.gloenvcha.2020.102168

None of the sources provides a comprehensive picture. The final energy demand ist of interest (without conversion losses in the energy supply chain).

In [ ]:
# final energy demands in kWh/pkm
intensity = pd.DataFrame(index=['air_domestic', 'air_international',
                                'bus', 'car', 'coach', 'rail_long', 'rail_short',
                                'non-motorised'],
                         data={'2020': [# From UBA TREMOD
                                        2.76/3.6,
                                        2.5/3.6,
                                        0.9/3.6,
                                        2.14/3.6,
                                        0.41/3.6,
                                        0.51/3.6,
                                        0.86/3.6,
                                        0],
                               '2050': [# From Robinius et al. (2020)
                                        2.76/3.6, # from today
                                        2.5/3.6, # from today
                                        1/10.626, # Battery-electric bus
                                        1/7.143 / params['car_occ']['all'], # BEV
                                        1/10.626, # Battery-electric bus
                                        1/23.419, # Electric train
                                        1/23.419, # Electric train
                                        0],
                               '2050_eff': [# From Millward-Hopkins et al. (2020)
                                            0.98/3.6,
                                            0.98/3.6,
                                            0.18/3.6,
                                            0.35/3.6,
                                            0.18/3.6,
                                            0.06/3.6,
                                            0.06/3.6,
                                            0]
                              })

In [ ]:
# energy demand with 2050 technology as bar plot
fig, ax = plt.subplots(figsize=(6,4))
demand = pd.DataFrame()
for s in scenarios:
    demand[s] = pkm_df[s] * intensity['2050']
demand = demand.fillna(0)
modes = [m for m in colormap.keys() if m in demand.index and m!='non-motorised']
demand.T[modes].plot.bar(stacked=True, ax=ax, legend=None,
                         color=[colormap[m] for m in modes])
ax.set_ylabel('PJ/a')
h,l = ax.get_legend_handles_labels()
plt.legend(h[::-1], l[::-1], loc='center left')
plt.savefig(output_path + '_scenario_validation/energy_demand_pkm.png', dpi=dpi)

In [ ]:
# Prepare zones to plot
zones = ref.zones.groupby('NUTS_ID').agg({'geometry': lambda g: g.unary_union,
                                          'population': 'sum', 'area': 'sum'})
zones = zones.set_crs(ref.epsg).to_crs(3857)

In [ ]:
# Map the traffic intensity
# Grey areas are not modelled due to random sampling of OD pairs
fig, ax = plt.subplots(nrows=1, ncols=len(scenarios)+1, figsize=(len(scenarios)*4*1.1, 5),
                       gridspec_kw={'width_ratios': [len(scenarios)/(1-0.1)]*len(scenarios)+[0.1]})
cbar_ref = None
for i in range(len(scenarios)):
    zones[scenarios[i]] = zones.merge(pkm[scenarios[i]].sum(axis=1).rename('pkm'),
                                      how='right', left_index=True, right_index=True)['pkm']
    plot = zones.plot(column=scenarios[i], colormap='Reds',
               missing_kwds={'color': 'lightgrey'},
               legend=True if i==0 else False,
               cax=ax[-1], legend_kwds={'label': "pkm per year"},
               ax=ax[i])
    ax[i].set_title(scenarios[i])
    ax[i].set_axis_off()
    #cbar = fig.colorbar(mappable=ScalarMappable(norm=None, cmap=get_cmap('Reds')),
    #                    cax=ax[-1])#, legend_kwds={'label': "pkm per year"})
    #if i==0:
    #    cbar_ref = cbar
    #else:
    #    cbar.mappable.set_clim(*cbar_ref.mappable.get_clim())

In [ ]:
# Map the traffic intensity divided by population
fig, ax = plt.subplots(nrows=1, ncols=len(scenarios)+1, figsize=(len(scenarios)*4*1.1, 5),
                       gridspec_kw={'width_ratios': [len(scenarios)/(1-0.1)]*len(scenarios)+[0.1]})
for i in range(len(scenarios)):
    zones[scenarios[i]] = zones.merge(pkm[scenarios[i]].sum(axis=1).rename('pkm'),
                                      how='right', left_index=True, right_index=True)['pkm']
    zones[scenarios[i]] /= zones['population']
    zones.plot(column=scenarios[i], colormap='Reds',
               missing_kwds={'color': 'lightgrey'},
               legend=True if i==0 else False,
               cax=ax[-1], legend_kwds={'label': "pkm per capita per year"},
               ax=ax[i])
    ax[i].set_title(scenarios[i])
    ax[i].set_axis_off()

### Street traffic load

In [ ]:
# Load assigned road networks
roads = {}
for s in scenarios:
    sm = stepmodel.read_zippedpickles(model_path + s + '/de_assignment')
    roads[s] = gpd.GeoDataFrame(sm.road_links.loc[sm.road_links['volume']>1], crs=ref.epsg)
    del sm

In [ ]:
# Map the road traffic
fig, ax = plt.subplots(nrows=1, ncols=len(scenarios), figsize=(len(scenarios)*4, 5))
scaler = 6e6
for i in range(len(scenarios)):
    roads[s] = roads[s].to_crs(zones.crs)
    roads[s].plot(alpha=.3, color='red', linewidth=list(roads[s]['volume']/scaler),
                  ax=zones.plot(alpha=.2, color='green', ax=ax[i]))
    ax[i].set_title(scenarios[i])
    ax[i].set_axis_off()

In [ ]:
# Average mileage
car_use = {'Average annual mileage': [], 'Private cars owned (mio.)': []}
for s in scenarios:
    params = excel.read_var(file='../input/parameters.xls', scenario=s)
    car_use['Average annual mileage'].append(
        np.round((roads[s]['volume']*roads[s]['length']/1000).sum()
                 / params['car_occ']['all'] / params['vehicles']['car'], -2))
    car_use['Private cars owned (mio.)'].append(np.round(params['vehicles']['car'] / 1e6, 1))
pd.DataFrame(index=scenarios, data=car_use)#.to_latex()